# Using a Neural Network for Medical Data Analysis


In this notebook, I'll be taking the data that I already cleaned and quantized in part one and will develop a neural network with hyperparameter tuning. Starting with the usual import calls. 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix, log_loss
import numpy as np
from keras.layers import Dense, Dropout
from keras import optimizers as opt
%matplotlib inline 

Calling the cleaned data. I was having some trouble with duplicate indicies, so I called the columns explicitly after reading in the file.

In [3]:
data = pd.read_csv('clean_data.csv').reset_index()
data = data[['slope_of_peak_exercise_st_segment','resting_blood_pressure',
      'chest_pain_type','num_major_vessels','fasting_blood_sugar_gt_120_mg_per_dl',
     'resting_ekg_results','serum_cholesterol_mg_per_dl','oldpeak_eq_st_depression','sex' 
     ,'age','max_heart_rate_achieved', 'exercise_induced_angina', 'heart_disease_present',
     'quant_thal']]

data.head()

,slope_of_peak_exercise_st_segment,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present,quant_thal
0,1,128,2,0,0,2,308,0.0,1,45,170,0,0,0
1,2,110,3,0,0,0,214,1.6,0,54,158,0,0,0
2,1,125,4,3,0,2,304,0.0,1,77,162,1,1,0
3,1,152,4,0,0,0,223,0.0,1,40,181,0,1,1
4,3,178,1,0,0,2,270,4.2,1,59,145,0,0,1


Separating the values and labels and using sklearn to standardize the dataset. 

In [4]:
vals = data.drop(columns = 'heart_disease_present')
labs = data['heart_disease_present']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X = vals)
scaled_vals = scaler.transform(vals)



Splitting the data into a training and testing set. 

In [5]:
from sklearn.model_selection import train_test_split
X = scaled_vals
y = labs

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

Now im going to use keras to define a quick model for testing. 

In [6]:
import keras
inputs = tf.keras.Input(shape = (13,))
x = layers.Dense(13, activation = 'softsign')(inputs)
#x = layers.BatchNormalization(axis = -1)(x)
x = layers.Dropout(rate = 0.1)(x)
#x = layers.BatchNormalization(axis = 1)(x)
#x = layers.Dense(8, activation = 'relu')(x)

predictions = layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)


model.compile( optimizer = 'Adam', learning_rate = 0.0025,
             loss = keras.losses.SparseCategoricalCrossentropy(),
             metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

#metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [7]:
X_train, y_train = np.asarray(X_train), np.asarray(y_train)



model.fit(X_train, y_train, batch_size = 64, epochs = 100, verbose = 1)

Train on 90 samples
Epoch 1/100
90/90 [==============================] - 1s 6ms/sample - loss: 0.7543 - sparse_categorical_accuracy: 0.4778
Epoch 2/100
90/90 [==============================] - 0s 0s/sample - loss: 0.7429 - sparse_categorical_accuracy: 0.4778
Epoch 3/100
90/90 [==============================] - 0s 44us/sample - loss: 0.7125 - sparse_categorical_accuracy: 0.5111
Epoch 4/100
90/90 [==============================] - 0s 89us/sample - loss: 0.7249 - sparse_categorical_accuracy: 0.5333
Epoch 5/100
90/90 [==============================] - 0s 44us/sample - loss: 0.6972 - sparse_categorical_accuracy: 0.5333
Epoch 6/100
90/90 [==============================] - 0s 44us/sample - loss: 0.7071 - sparse_categorical_accuracy: 0.5556
Epoch 7/100
90/90 [==============================] - 0s 89us/sample - loss: 0.6751 - sparse_categorical_accuracy: 0.6222
Epoch 8/100
90/90 [==============================] - 0s 44us/sample - loss: 0.6703 - sparse_categorical_accuracy: 0.5889
Epoch 9/100
90/

90/90 [==============================] - 0s 89us/sample - loss: 0.3908 - sparse_categorical_accuracy: 0.8444
Epoch 69/100
90/90 [==============================] - 0s 44us/sample - loss: 0.4085 - sparse_categorical_accuracy: 0.8556
Epoch 70/100
90/90 [==============================] - 0s 44us/sample - loss: 0.4114 - sparse_categorical_accuracy: 0.8333
Epoch 71/100
90/90 [==============================] - 0s 89us/sample - loss: 0.3956 - sparse_categorical_accuracy: 0.8444
Epoch 72/100
90/90 [==============================] - 0s 44us/sample - loss: 0.3937 - sparse_categorical_accuracy: 0.8556
Epoch 73/100
90/90 [==============================] - 0s 89us/sample - loss: 0.3858 - sparse_categorical_accuracy: 0.8667
Epoch 74/100
90/90 [==============================] - 0s 89us/sample - loss: 0.3845 - sparse_categorical_accuracy: 0.8556
Epoch 75/100
90/90 [==============================] - 0s 89us/sample - loss: 0.3942 - sparse_categorical_accuracy: 0.8556
Epoch 76/100
90/90 [=================

The returned predictions are one-hot encoded, for two possibile values for heart disease: present or not present. To call sklearn's metrics, they need to be converted to a simple array with either 1 or 0. I'll define a simple function to do this. 

In [8]:
def convert(preds):
    new_preds = []
    for pred in preds:
        if pred[0] > pred[1]:
            new_preds.append(0)
        else:
            new_preds.append(1)
    return new_preds

In [9]:
import numpy as np
X_test = np.asarray(X_test)
preds = model.predict(X_test)
preds = convert(preds)

print(classification_report(y_test,preds))
print('log loss: ',log_loss(y_test, preds))

              precision    recall  f1-score   support

           0       0.77      0.83      0.80        48
           1       0.79      0.71      0.75        42

    accuracy                           0.78        90
   macro avg       0.78      0.77      0.78        90
weighted avg       0.78      0.78      0.78        90

log loss:  7.675354718640613


Of course these results are not ideal, lets see if a gridsearch of hyper-parameters will give any improvements.

# Grid Searching Hyper Parameters 

In [10]:
from keras.models import Sequential
from keras import optimizers as opt
from keras.wrappers.scikit_learn import KerasClassifier


Below I'm defining all of the parameters I may want to test

In [11]:
from sklearn.model_selection import GridSearchCV


batch_size = [50,60,80,100]
epochs = [10,40,80,100]
num_neurons = [8,16,32,64,128]
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
#optimizer = [opt.Adam, opt.SGD, opt.RMSprop, opt.Adagrad, opt.Adadelta, opt.Adamax, opt.Nadam]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2]





Doing a test for a 'Fast grid' to make sure code is working properly

In [12]:
def create_seq_model_fast():

    model = Sequential()

    model.add(Dense(16, input_dim = 13, activation = 'softsign'))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation = 'softmax'))
    model.compile(optimizer= opt.Adam(), loss = keras.losses.SparseCategoricalCrossentropy(),
                  metrics = ['accuracy'])

    return model


    
param_grid_fast = {'batch_size': batch_size, 'epochs': epochs}  
model = KerasClassifier(build_fn=create_seq_model_fast, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid_fast, n_jobs=-1, cv = 3)

grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

df = pd.DataFrame(grid_result.cv_results_)



Best: 0.833333 using {'batch_size': 100, 'epochs': 80}


Now for the actual model

In [13]:
def create_seq_model(activation, optimizer, init_mode, dropout_rate):

    model = Sequential()

    model.add(Dense(16, input_dim = 13, activation = activation, kernel_initializer = init_mode))
    model.add(Dropout(dropout_rate))
    model.add(Dense(2, activation = 'softmax', kernel_initializer = init_mode))
    model.compile(optimizer= optimizer, loss = keras.losses.SparseCategoricalCrossentropy(),
                  metrics = ['accuracy'])

    return model

In [ ]:
model = KerasClassifier(build_fn=create_seq_model, epochs = 100, batch_size = 50, verbose=0)
'''
param_grid = {'batch_size' : batch_size, 'epochs' : epochs, 
              'num_neurons':num_neurons, 'activation': activation, 
              'optimizer': optimizer, 'init_mode': init_mode, 'dropout_rate': dropout_rate,
             'weight_constraint': weight_constraint}
'''

param_grid = {'activation': activation, 
              'optimizer': optimizer, 'init_mode': init_mode, 'dropout_rate': dropout_rate,}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-2, cv = 3, verbose = 2, scoring = 'accuracy')

grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

df = pd.DataFrame(grid_result.cv_results_)
df.to_csv('grid_results_accuracy.csv')



Fitting 3 folds for each of 1344 candidates, totalling 4032 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-2)]: Done 351 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-2)]: Done 634 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-2)]: Done 999 tasks      | elapsed:  5.3min
